# Frequent Pattern Mining in Spark


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.fpm import FPGrowth

Let's initialize the Spark context.

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [4]:
products = spark.read.csv('files/products.csv', header=True, inferSchema=True)
orders = spark.read.csv('files/order_products__train.csv', header=True, inferSchema=True)

In [5]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)


In [6]:
#rename product_id to product_id2
orders =  orders.withColumnRenamed("product_id", "product_id2")
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id2: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)


Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [7]:
''' 2 lines of code expected '''
# YOUR CODE HERE
join = orders.join(products, orders.product_id2 == products.product_id)
df = join.groupBy("order_id").agg(collect_list("product_id2").alias("products"))
df.show()
df.printSchema()

+--------+--------------------+
|order_id|            products|
+--------+--------------------+
|     762|[21137, 41220, 15...|
|     844|[14992, 21405, 11...|
|     988|[45061, 28464, 12...|
|    1139|[24852, 21137, 34...|
|    1143|[42719, 42097, 36...|
|    1280|[48186, 49235, 23...|
|    1342|[13176, 30827, 14...|
|    1350|[12260, 25042, 38...|
|    1468|[24852, 46667, 30...|
|    1591|[17203, 44008, 48...|
|    1721|[38689, 27845, 22...|
|    1890|[432, 47626, 1317...|
|    1955|[10132, 13176, 16...|
|    2711|[9387, 17122, 273...|
|    2888|[30450, 41486, 46...|
|    3010|[2966, 47209, 131...|
|    3037|[19348, 19173, 6994]|
|    3179|[35221, 39322, 36...|
|    4036|[11758, 46979, 48...|
|    4092|[26209, 6615, 445...|
+--------+--------------------+
only showing top 20 rows

root
 |-- order_id: integer (nullable = true)
 |-- products: array (nullable = false)
 |    |-- element: integer (containsNull = false)


In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with

```minSupport=0.01``` and ```minConfidence=0.5```

In [8]:
''' 3 lines of code expected '''
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(df)

Compute how many frequent itemsets and association rules were generated by running FP-growth alongside
visualizing top frequent itemsets and association rules.


In [9]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
#Order by freq descending
model.freqItemsets.orderBy("freq", ascending=False).show()
print("Number of frequent itemsets: " + str(model.freqItemsets.count()))

+-------+-----+
|  items| freq|
+-------+-----+
|[24852]|18726|
|[13176]|15480|
|[21137]|10894|
|[21903]| 9784|
|[47626]| 8135|
|[47766]| 7409|
|[47209]| 7293|
|[16797]| 6494|
|[26209]| 6033|
|[27966]| 5546|
|[39275]| 4966|
|[27845]| 4908|
|[30391]| 4613|
|[45007]| 4589|
|[22935]| 4290|
|[24964]| 4158|
| [4920]| 4059|
|[46979]| 3868|
|[40706]| 3823|
| [8518]| 3818|
+-------+-----+
Number of frequent itemsets: 120


In [10]:
#Print out association rules and count
model.associationRules.show()
print("Number of association rules: " + str(model.associationRules.count()))

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+

Number of association rules: 0


Now retrain the FP-growth model changing only
```minsupport=0.001```
and compute how many frequent itemsets and association rules were generated.

In [11]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(df)

In [12]:
model.freqItemsets.orderBy("freq", ascending=False).show()
print("Number of frequent itemsets: " + str(model.freqItemsets.count()))

+-------+-----+
|  items| freq|
+-------+-----+
|[24852]|18726|
|[13176]|15480|
|[21137]|10894|
|[21903]| 9784|
|[47626]| 8135|
|[47766]| 7409|
|[47209]| 7293|
|[16797]| 6494|
|[26209]| 6033|
|[27966]| 5546|
|[39275]| 4966|
|[27845]| 4908|
|[30391]| 4613|
|[45007]| 4589|
|[22935]| 4290|
|[24964]| 4158|
| [4920]| 4059|
|[46979]| 3868|
|[40706]| 3823|
| [8518]| 3818|
+-------+-----+
Number of frequent itemsets: 4444


In [13]:
model.associationRules.show()
print("Number of association rules: " + str(model.associationRules.count()))

+--------------------+----------+------------------+------------------+--------------------+
|          antecedent|consequent|        confidence|              lift|             support|
+--------------------+----------+------------------+------------------+--------------------+
|       [4605, 16797]|   [24852]|0.5357142857142857|3.7536332219526702|0.001143214261216...|
|[30391, 47209, 21...|   [13176]|          0.546875| 4.635330870478036|0.001066999977135...|
|      [22825, 47209]|   [13176]|0.5170454545454546|4.3824946411792345|0.001387099970276...|
|       [9839, 47209]|   [13176]|0.5048231511254019| 4.278897986822536|0.001196564260073623|
|      [39928, 47209]|   [13176]|0.5459770114942529| 4.627719489738336|0.001448071397541327|
|       [8174, 27966]|   [13176]|0.5412186379928315| 4.587387356098284|0.001150835689624...|
|      [22035, 47209]|   [13176]|0.5314685314685315| 4.504745125675359|0.001158457118033...|
|      [27966, 47209]|   [13176]| 0.521099116781158| 4.416853618458589